In [134]:
import pathlib
import datetime
import warnings

from tqdm.notebook import tqdm

import shapely.geometry
import geopandas as gpd
import pandas as pd
import numpy as np
import movingpandas as mpd

warnings.filterwarnings('ignore')

In [178]:
data_dir = pathlib.Path('~/data/drought/ivs').expanduser()
processed_gdf = gpd.read_file(data_dir / 'trips.geojson').set_crs('EPSG:4326')

In [179]:
processed_gdf['distance'] = processed_gdf['geometry'].to_crs('EPSG:32631').length
processed_utm31n_gdf = processed_gdf.to_crs('EPSG:32631')


In [180]:
processed_gdf['reizen'].sum()

3169

In [189]:
trips = []

extra_columns = ["source_country", "source_city", "target_country", "target_city", "jaar", "week"]

traj_id = 0 
for week_row_i, route_week_row in tqdm(processed_gdf.iterrows()):
    geom = route_week_row.geometry
    projected_geom = processed_utm31n_gdf.loc[week_row_i].geometry
    sample_days = np.random.uniform(0, 7, size=route_week_row['reizen']) 
    speeds = np.random.uniform(3, 4, size=route_week_row['reizen']) 
    extra_info = {key: val for (key, val) in route_week_row.items() if key in extra_columns}
    for trip_i in range(route_week_row['reizen']):
        traj_id += 1
        date_str = f"{route_week_row['jaar']}W{route_week_row['week']:02d}-0"
        t0 = datetime.datetime.strptime(date_str, "%YW%U-%w")
        t0 = t0 + datetime.timedelta(days=sample_days[trip_i])
        speed = speeds[trip_i]
        duration = route_week_row['distance'] / speed # m * m/s
        t1 = t0 + datetime.timedelta(seconds=duration)
        trips.append({
            "traj_id": traj_id,
            # trip number of this week
            "trip_id": trip_i,
            "t0": t0,
            "t1": t1,
            "duration": duration,
            "speed": speed,
            "geometry": geom,
            "geometry_utm31n": projected_geom,
            "lading": route_week_row['lading'] / route_week_row['reizen'],
            **extra_info
        })
trips_gdf = gpd.GeoDataFrame(
    pd.DataFrame(trips).set_index('traj_id')
)
trips_gdf

0it [00:00, ?it/s]

,trip_id,t0,t1,duration,speed,geometry,geometry_utm31n,lading,source_country,source_city,target_country,target_city,jaar,week
traj_id,,,,,,,,,,,,,,
1,0,2022-01-05 03:40:23.120264,2022-01-05 14:35:46.274721,39323.154457,3.563394,"LINESTRING (4.09117 51.97514, 4.07253 51.98162...",LINESTRING (574949.7221097306 5758835.31840901...,719.0,NL,AMS,BE,ANR,2022,1
2,1,2022-01-03 05:10:36.756338,2022-01-03 14:59:37.755724,35340.999386,3.964910,"LINESTRING (4.09117 51.97514, 4.07253 51.98162...",LINESTRING (574949.7221097306 5758835.31840901...,719.0,NL,AMS,BE,ANR,2022,1
3,2,2022-01-04 22:48:28.912243,2022-01-05 09:11:28.440949,37379.528706,3.748680,"LINESTRING (4.09117 51.97514, 4.07253 51.98162...",LINESTRING (574949.7221097306 5758835.31840901...,719.0,NL,AMS,BE,ANR,2022,1
4,3,2022-01-02 00:32:13.078505,2022-01-02 10:18:34.150700,35181.072195,3.982934,"LINESTRING (4.09117 51.97514, 4.07253 51.98162...",LINESTRING (574949.7221097306 5758835.31840901...,719.0,NL,AMS,BE,ANR,2022,1
5,4,2022-01-06 19:36:35.365053,2022-01-07 07:15:13.198200,41917.833147,3.342823,"LINESTRING (4.09117 51.97514, 4.07253 51.98162...",LINESTRING (574949.7221097306 5758835.31840901...,719.0,NL,AMS,BE,ANR,2022,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3165,1,2022-09-13 06:10:37.962826,2022-09-15 08:07:04.421836,179786.459010,3.359746,"LINESTRING (4.09117 51.97514, 4.07253 51.98162...",LINESTRING (574949.7221097306 5758835.31840901...,1330.0,NL,AMS,DE,LUH,2022,37
3166,0,2022-09-24 03:35:27.562785,2022-09-26 07:11:15.237823,185747.675038,3.251921,"LINESTRING (4.09117 51.97514, 4.07253 51.98162...",LINESTRING (574949.7221097306 5758835.31840901...,1274.0,NL,AMS,DE,LUH,2022,38
3167,1,2022-09-22 21:40:27.091976,2022-09-25 04:06:23.902276,195956.810300,3.082500,"LINESTRING (4.09117 51.97514, 4.07253 51.98162...",LINESTRING (574949.7221097306 5758835.31840901...,1274.0,NL,AMS,DE,LUH,2022,38


In [195]:
trips_gdf.drop(columns=['geometry_utm31n']).to_file(data_dir / 'trip_metadata.geojson')

In [190]:
rows = []
for traj_id, trip_row in tqdm(trips_gdf.iterrows(), total=trips_gdf.shape[0]):
    projected_geom = trip_row['geometry_utm31n']
    geom = trip_row['geometry']
    t0 = trip_row['t0']
    speed = trip_row['speed']
    for i in range(len(projected_geom.coords)):
        row = {}
        if i == 0:
            # no points
            cum_distance = 0
        else: 
            # index 1 -> select points 0:2 (0, 1)
            line = shapely.geometry.LineString(projected_geom.coords[:(i + 1)])
            cum_distance = line.length

        point = shapely.geometry.Point(geom.coords[i])
        duration = cum_distance / speed
        t = t0 + datetime.timedelta(seconds=duration)
        rows.append({
            'traj': traj_id,
            't': t,
            'geometry': point,
        })


  0%|          | 0/3169 [00:00<?, ?it/s]

In [191]:
traj_gdf = gpd.GeoDataFrame(
    pd.DataFrame(rows)
).set_crs('EPSG:4326')

In [193]:
traj_gdf.to_file(data_dir / 'traj.geojson')